In [3]:
import numpy as np
from pathlib import Path
import sys

In [1]:
class TreeNode:
    def __init__(self, level, V, W, taken):
        self.level = level # niveau dans l’arbre de recherche >=0
        self.V = V # valeur de la solution courante
        self.W = W # poids de la solution courante
        self.taken = taken # liste des index d’objets de la solution courante
    def __str__(self):
        return str((self.level, self.V, self.W, self.taken))

In [23]:
class KPBB:
    def __init__(self, cap, values, weights,taille_pop,coef_filtre,pourcent): # Initialisation des données
        self.capacity = cap
        self.values = values
        self.weights = weights
        self.unordered = [((v, w), i) for i, (v, w) in enumerate(zip(self.values, self.weights))]
        self.ordered = sorted([((v, w), i) for i, (v, w) in enumerate(zip(self.values, self.weights))], key = lambda tup: float(tup[0][0])/tup[0][1], reverse = True)
        self.taille_pop=taille_pop
        self.coef_filtre=coef_filtre
        self.proba=pourcent
        self.population=[]
        self.epoch=0
    def genere_pop(self):
        if  len(self.population)==0:
            for i in self.taille_pop:
                individus=[np.random.randint(0,1) for i in range(len(self.unordered))]
            while np.sum(self.weights[individus])>self.capacity:
                individus=[np.random.randint(0,1) for i in range(len(self.unordered))]
            self.population.append(individus)
        else:
           self.mutation() 

    def filtre(self):
        value=[np.sum(self.values[p]) for p in self.population]
        value=np.argsort(value)
        nb_filtre=(self.coef_filtre*self.taille_pop)/100
        self.population=self.population[nb_filtre:]
    
    def cross_over(self):
        index=np.random.randint(0,self.taille_pop,2)
        parent=self.population[index]
        taille_div=np.random.randint(1,self.taille_pop)
        nb_genere=100-((self.coef_filtre*self.taille_pop)/100)
        childs=[]
        for i in range(nb_genere):
            index=np.random.randint(0,self.taille_pop,2)
            parent=self.population[index]
            taille_div=np.random.randint(1,self.taille_pop)  
            child=np.concatenate([parent[0][0:taille_div],parent[1][taille_div:]])
            childs.append(child)
        return childs

        
    def mutation(self):
        childs=self.cross_over()
        p=np.random.uniform(0,1)
        for child in childs:
            p=np.random.uniform(0,1)
            if p>=self.proba:
                index=np.random.randint(0,len(self.values))
                child[index]=1-child[index]
            self.population.append(child)
    def solv(self):
        for i in range(self.epoch):
            self.population()
            self.cross_over()
            self.mutation()
            self.population()
        return self.population